## 네이버 검색 API크롤링

In [12]:
import os
import urllib.request
import datetime
import json

import pandas as pd
import html
import re
from dotenv import load_dotenv
load_dotenv(verbose=True)

client_id = os.getenv('CLI_ID')
client_secret = os.getenv('CLI_SEC')

In [13]:
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [14]:
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [15]:
def getPostData(post, jsonResult, search):
    title = post['title']
    description = post['description']
    link = post['link']
    

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'search':search, 'title': title, 'description': description,
                       'link': link, 'pub_date': pDate})

In [16]:
def clean_text(text):
    text = html.unescape(text)
    text = re.sub(r'<[^>]+>', '', text)
    return text

In [17]:
node = 'news'
jsonResult = []

# srch1 = input('검색어1: ')
# srch2 = input('검색어2: ')
search_topic = ['AI', '부동산']

for srchText in search_topic:
    jsonResponse = getNaverSearch(node, srchText, 1, 100)    

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            getPostData(post, jsonResult, srchText) 

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srchText, start, 100)


df = pd.DataFrame(jsonResult)
df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)

[2024-05-26 19:25:15.315900] Url Request Success
[2024-05-26 19:25:15.489890] Url Request Success
[2024-05-26 19:25:15.719404] Url Request Success
[2024-05-26 19:25:15.907047] Url Request Success
[2024-05-26 19:25:16.208110] Url Request Success
[2024-05-26 19:25:16.468613] Url Request Success
[2024-05-26 19:25:16.722189] Url Request Success
[2024-05-26 19:25:16.935109] Url Request Success
[2024-05-26 19:25:17.149495] Url Request Success
[2024-05-26 19:25:17.372245] Url Request Success
HTTP Error 400: Bad Request
[2024-05-26 19:25:17.480964] Error for URL : https://openapi.naver.com/v1/search/news.json?query=AI&start=1001&display=100
[2024-05-26 19:25:17.647083] Url Request Success
[2024-05-26 19:25:17.828498] Url Request Success
[2024-05-26 19:25:18.052021] Url Request Success
[2024-05-26 19:25:18.299838] Url Request Success
[2024-05-26 19:25:18.540240] Url Request Success
[2024-05-26 19:25:18.742586] Url Request Success
[2024-05-26 19:25:19.040771] Url Request Success
[2024-05-26 19:2

In [18]:
nnews_df = df[df['link'].str.contains('https://n.news.naver.com/', na=False)]
nnews_df.reset_index(drop=True, inplace=True)
nnews_df

,search,title,description,link,pub_date
0,AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI,지능 AI가 산업 현장 뿐 아니라 문화 예술에서도 활용되고 있지요. 이제는 웹툰도 ...,https://n.news.naver.com/mnews/article/448/000...,2024-05-26 19:19:00
1,AI,"팻 겔싱어 인텔 CEO, 6월 방한 '무산'",26일 업계에 따르면 겔싱어 CEO는 내달 5일 인텔코리아가 서울에서 진행하는 'A...,https://n.news.naver.com/mnews/article/003/001...,2024-05-26 19:17:00
2,AI,고준위 특별법-모성3법 등 민생-경제 법안 줄폐기 ‘빨간불’,"여야가 ‘채 상병 특검법’ 재표결, 연금개혁안 처리 문제 등으로 대치하면서 ‘고준위...",https://n.news.naver.com/mnews/article/020/000...,2024-05-26 19:12:00
3,AI,위기의 삼성 반도체…초격차 변화 예고,삼성전자가 인공지능(AI) 반도체의 핵심인 고대역폭메모리(HBM)와 파운드리(반도체...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:11:00
4,AI,"안철수 ""인문학 AI로 古書 복원하면 K콘텐츠 만들 소재 쏟아질 것""",인공지능(AI)으로 모두 번역할 수 있으면 이를 기반으로 영화·드라마 등 뛰어난 K...,https://n.news.naver.com/mnews/article/015/000...,2024-05-26 19:10:00
...,...,...,...,...,...
930,부동산,"[올댓차이나] '자금난' 中 완커기업, 협조융자로 3.7조원 자금 조달",극심한 자금난으로 디폴트(채무불이행) 위기에 직면한 중국 2위 부동산 개발사 완커기...,https://n.news.naver.com/mnews/article/003/001...,2024-05-24 14:41:00
931,부동산,“반갑다” 지역 첫 공급 브랜드 아파트 눈길,24일 한국부동산원 청약홈에 따르면 지난해 12월 경기 부천에서 첫 선을 보인 ‘롯...,https://n.news.naver.com/mnews/article/009/000...,2024-05-24 14:39:00
932,부동산,수도권 아파트 '우상향' 시대… '평택 화양 동문 디 이스트',실제로 KB부동산 주택가격동향 자료를 분석한 결과 1999년 통계 공개 이래 수도권...,https://n.news.naver.com/mnews/article/119/000...,2024-05-24 14:39:00
933,부동산,"""너무 비쌌나""…LH, 여의도 부지 또 유찰",평당가도 1억6000만원 수준이라 그에 맞는 수익성이 나는 부동산상품으로 개발하기 ...,https://n.news.naver.com/mnews/article/029/000...,2024-05-24 14:38:00


In [26]:
value1_subset = nnews_df[nnews_df['search'] == 'AI'].head(5)
value2_subset = nnews_df[nnews_df['search'] == '부동산'].head(5)

# 두 개의 서브셋을 합침
nnews_df = pd.concat([value1_subset, value2_subset])
nnews_df.reset_index(drop=True, inplace=True)
nnews_df

,search,title,description,link,pub_date
0,AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI,지능 AI가 산업 현장 뿐 아니라 문화 예술에서도 활용되고 있지요. 이제는 웹툰도 ...,https://n.news.naver.com/mnews/article/448/000...,2024-05-26 19:19:00
1,AI,"팻 겔싱어 인텔 CEO, 6월 방한 '무산'",26일 업계에 따르면 겔싱어 CEO는 내달 5일 인텔코리아가 서울에서 진행하는 'A...,https://n.news.naver.com/mnews/article/003/001...,2024-05-26 19:17:00
2,AI,고준위 특별법-모성3법 등 민생-경제 법안 줄폐기 ‘빨간불’,"여야가 ‘채 상병 특검법’ 재표결, 연금개혁안 처리 문제 등으로 대치하면서 ‘고준위...",https://n.news.naver.com/mnews/article/020/000...,2024-05-26 19:12:00
3,AI,위기의 삼성 반도체…초격차 변화 예고,삼성전자가 인공지능(AI) 반도체의 핵심인 고대역폭메모리(HBM)와 파운드리(반도체...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:11:00
4,AI,"안철수 ""인문학 AI로 古書 복원하면 K콘텐츠 만들 소재 쏟아질 것""",인공지능(AI)으로 모두 번역할 수 있으면 이를 기반으로 영화·드라마 등 뛰어난 K...,https://n.news.naver.com/mnews/article/015/000...,2024-05-26 19:10:00
5,부동산,[안종범의 정책진단] 부자증세라는 포퓰리즘,버는 것은 없으면서 부동산이나 예금과 주식 같은 금융자산만 많이 가진 자도 부자인가...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:15:00
6,부동산,"민주 ""'구하라법' 등 민생법안 처리 위해 법사위 열자""","이어 ""코로나 팬데믹 기간 주로 시골 지역의 부동산 권리행사를 할 수 없었던 많은 ...",https://n.news.naver.com/mnews/article/031/000...,2024-05-26 19:06:00
7,부동산,"정부 평가기준 내놓자 바로 채점표 등장... 분당·평촌 ""결과 지켜보자"" 거래...","분당 한솔마을 인근 공인중개사는 ""1기 신도시 선도지구 발표에도 이 지역 부동산 분...",https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:04:00
8,부동산,"美 연착륙 할까, M7 넘을 종목은…'투자의 심장' 뉴욕서 해답 찾는다","KKR은 부동산 등에 총 4290억달러를 운용하고 있어 자산별, 지역별 시장 상황을...",https://n.news.naver.com/mnews/article/015/000...,2024-05-26 19:02:00
9,부동산,중소·중견기업 '가업 승계' 위해 상속·증여 공제 늘리나,다만 '부의 대물림'을 심화시킬 수 있는 부동산 등 일반 자산의 이전에는 칸막이를 ...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:01:00


In [20]:
nnews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 0 to 466
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   search       60 non-null     object
 1   title        60 non-null     object
 2   description  60 non-null     object
 3   link         60 non-null     object
 4   pub_date     60 non-null     object
dtypes: object(5)
memory usage: 2.8+ KB


In [27]:
nnews_df['link'].iloc[0]

'https://n.news.naver.com/mnews/article/021/0002639037?sid=102'

In [27]:
test_df = nnews_df.copy()
test_df

,search,title,description,link,pub_date
0,AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI,지능 AI가 산업 현장 뿐 아니라 문화 예술에서도 활용되고 있지요. 이제는 웹툰도 ...,https://n.news.naver.com/mnews/article/448/000...,2024-05-26 19:19:00
1,AI,"팻 겔싱어 인텔 CEO, 6월 방한 '무산'",26일 업계에 따르면 겔싱어 CEO는 내달 5일 인텔코리아가 서울에서 진행하는 'A...,https://n.news.naver.com/mnews/article/003/001...,2024-05-26 19:17:00
2,AI,고준위 특별법-모성3법 등 민생-경제 법안 줄폐기 ‘빨간불’,"여야가 ‘채 상병 특검법’ 재표결, 연금개혁안 처리 문제 등으로 대치하면서 ‘고준위...",https://n.news.naver.com/mnews/article/020/000...,2024-05-26 19:12:00
3,AI,위기의 삼성 반도체…초격차 변화 예고,삼성전자가 인공지능(AI) 반도체의 핵심인 고대역폭메모리(HBM)와 파운드리(반도체...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:11:00
4,AI,"안철수 ""인문학 AI로 古書 복원하면 K콘텐츠 만들 소재 쏟아질 것""",인공지능(AI)으로 모두 번역할 수 있으면 이를 기반으로 영화·드라마 등 뛰어난 K...,https://n.news.naver.com/mnews/article/015/000...,2024-05-26 19:10:00
5,부동산,[안종범의 정책진단] 부자증세라는 포퓰리즘,버는 것은 없으면서 부동산이나 예금과 주식 같은 금융자산만 많이 가진 자도 부자인가...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:15:00
6,부동산,"민주 ""'구하라법' 등 민생법안 처리 위해 법사위 열자""","이어 ""코로나 팬데믹 기간 주로 시골 지역의 부동산 권리행사를 할 수 없었던 많은 ...",https://n.news.naver.com/mnews/article/031/000...,2024-05-26 19:06:00
7,부동산,"정부 평가기준 내놓자 바로 채점표 등장... 분당·평촌 ""결과 지켜보자"" 거래...","분당 한솔마을 인근 공인중개사는 ""1기 신도시 선도지구 발표에도 이 지역 부동산 분...",https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:04:00
8,부동산,"美 연착륙 할까, M7 넘을 종목은…'투자의 심장' 뉴욕서 해답 찾는다","KKR은 부동산 등에 총 4290억달러를 운용하고 있어 자산별, 지역별 시장 상황을...",https://n.news.naver.com/mnews/article/015/000...,2024-05-26 19:02:00
9,부동산,중소·중견기업 '가업 승계' 위해 상속·증여 공제 늘리나,다만 '부의 대물림'을 심화시킬 수 있는 부동산 등 일반 자산의 이전에는 칸막이를 ...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:01:00


## 셀레늄 크롤링

In [23]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action='ignore')

In [24]:
def content_crawl(url_path):
    chrome_options = webdriver.ChromeOptions()
    #chrome_options.add_argument('--headless')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url_path)
    time.sleep(3)
    article_element = driver.find_element(By.XPATH, '//*[@id="dic_area"]')
    content = article_element.text
    return content


In [28]:
test_df['article'] = test_df['link'].apply(lambda x: content_crawl(x))
test_df

,search,title,description,link,pub_date,article
0,AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI,지능 AI가 산업 현장 뿐 아니라 문화 예술에서도 활용되고 있지요. 이제는 웹툰도 ...,https://n.news.naver.com/mnews/article/448/000...,2024-05-26 19:19:00,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI\nTV조선 뉴스\nPlay...
1,AI,"팻 겔싱어 인텔 CEO, 6월 방한 '무산'",26일 업계에 따르면 겔싱어 CEO는 내달 5일 인텔코리아가 서울에서 진행하는 'A...,https://n.news.naver.com/mnews/article/003/001...,2024-05-26 19:17:00,[서울=뉴시스]인텔 비전 2024에서 팻 겔싱어 인텔 CEO가 인텔 가우디 3 가속...
2,AI,고준위 특별법-모성3법 등 민생-경제 법안 줄폐기 ‘빨간불’,"여야가 ‘채 상병 특검법’ 재표결, 연금개혁안 처리 문제 등으로 대치하면서 ‘고준위...",https://n.news.naver.com/mnews/article/020/000...,2024-05-26 19:12:00,ⓒ News1\n21대 국회 임기 만료가 3일 남은 가운데 여야는 민생-경제 법안...
3,AI,위기의 삼성 반도체…초격차 변화 예고,삼성전자가 인공지능(AI) 반도체의 핵심인 고대역폭메모리(HBM)와 파운드리(반도체...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:11:00,새수장 앞세워 대대적 전략 수정\n엔비디아 검증 탈락설 즉각 반박\n삼성전자가 인공...
4,AI,"안철수 ""인문학 AI로 古書 복원하면 K콘텐츠 만들 소재 쏟아질 것""",인공지능(AI)으로 모두 번역할 수 있으면 이를 기반으로 영화·드라마 등 뛰어난 K...,https://n.news.naver.com/mnews/article/015/000...,2024-05-26 19:10:00,22대 국회 이것만큼은\n안철수 국민의힘 분당갑 당선인\n\n최근 복원된 승정원일기...
5,부동산,[안종범의 정책진단] 부자증세라는 포퓰리즘,버는 것은 없으면서 부동산이나 예금과 주식 같은 금융자산만 많이 가진 자도 부자인가...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:15:00,부자 정의 명확지 않은채\n무조건 ‘돈 더 많이 내라’\n세금 초과 부담 최소화를\...
6,부동산,"민주 ""'구하라법' 등 민생법안 처리 위해 법사위 열자""","이어 ""코로나 팬데믹 기간 주로 시골 지역의 부동산 권리행사를 할 수 없었던 많은 ...",https://n.news.naver.com/mnews/article/031/000...,2024-05-26 19:06:00,"""단 10건이라도 처리해야…국회의 당연한 책무""\n더불어민주당이 26일 국민의힘을 ..."
7,부동산,"정부 평가기준 내놓자 바로 채점표 등장... 분당·평촌 ""결과 지켜보자"" 거래...","분당 한솔마을 인근 공인중개사는 ""1기 신도시 선도지구 발표에도 이 지역 부동산 분...",https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:04:00,선도지구 선정 앞둔 1기 신도시\n소형-대형 면적따라 유불리 갈려\n재건축 주민동의...
8,부동산,"美 연착륙 할까, M7 넘을 종목은…'투자의 심장' 뉴욕서 해답 찾는다","KKR은 부동산 등에 총 4290억달러를 운용하고 있어 자산별, 지역별 시장 상황을...",https://n.news.naver.com/mnews/article/015/000...,2024-05-26 19:02:00,6월 3~5일 한경 글로벌마켓 콘퍼런스 NYC 2024\n\n맥베이 KKR CIO가...
9,부동산,중소·중견기업 '가업 승계' 위해 상속·증여 공제 늘리나,다만 '부의 대물림'을 심화시킬 수 있는 부동산 등 일반 자산의 이전에는 칸막이를 ...,https://n.news.naver.com/mnews/article/014/000...,2024-05-26 19:01:00,"정부, 성장사다리 대책 추진\n승계 관련 세제 포함도 검토\n부동산 등 자산 이전은..."


## 데이터 적재

In [35]:
test_df = test_df[['search','title','article','pub_date']]
test_df.columns = ['검색어', '제목', '내용', '발행일']
test_df

,검색어,제목,내용,발행일
0,AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI\nTV조선 뉴스\nPlay...,2024-05-26 19:19:00
1,AI,"팻 겔싱어 인텔 CEO, 6월 방한 '무산'",[서울=뉴시스]인텔 비전 2024에서 팻 겔싱어 인텔 CEO가 인텔 가우디 3 가속...,2024-05-26 19:17:00
2,AI,고준위 특별법-모성3법 등 민생-경제 법안 줄폐기 ‘빨간불’,ⓒ News1\n21대 국회 임기 만료가 3일 남은 가운데 여야는 민생-경제 법안...,2024-05-26 19:12:00
3,AI,위기의 삼성 반도체…초격차 변화 예고,새수장 앞세워 대대적 전략 수정\n엔비디아 검증 탈락설 즉각 반박\n삼성전자가 인공...,2024-05-26 19:11:00
4,AI,"안철수 ""인문학 AI로 古書 복원하면 K콘텐츠 만들 소재 쏟아질 것""",22대 국회 이것만큼은\n안철수 국민의힘 분당갑 당선인\n\n최근 복원된 승정원일기...,2024-05-26 19:10:00
5,부동산,[안종범의 정책진단] 부자증세라는 포퓰리즘,부자 정의 명확지 않은채\n무조건 ‘돈 더 많이 내라’\n세금 초과 부담 최소화를\...,2024-05-26 19:15:00
6,부동산,"민주 ""'구하라법' 등 민생법안 처리 위해 법사위 열자""","""단 10건이라도 처리해야…국회의 당연한 책무""\n더불어민주당이 26일 국민의힘을 ...",2024-05-26 19:06:00
7,부동산,"정부 평가기준 내놓자 바로 채점표 등장... 분당·평촌 ""결과 지켜보자"" 거래...",선도지구 선정 앞둔 1기 신도시\n소형-대형 면적따라 유불리 갈려\n재건축 주민동의...,2024-05-26 19:04:00
8,부동산,"美 연착륙 할까, M7 넘을 종목은…'투자의 심장' 뉴욕서 해답 찾는다",6월 3~5일 한경 글로벌마켓 콘퍼런스 NYC 2024\n\n맥베이 KKR CIO가...,2024-05-26 19:02:00
9,부동산,중소·중견기업 '가업 승계' 위해 상속·증여 공제 늘리나,"정부, 성장사다리 대책 추진\n승계 관련 세제 포함도 검토\n부동산 등 자산 이전은...",2024-05-26 19:01:00


In [40]:
import uuid

## 아이디, 요약 컬럼 추가
test_df['아이디'] = [str(uuid.uuid4()) for _ in range(len(test_df))]
test_df['요약'] = ''

In [41]:
test_df

,검색어,제목,내용,발행일,아이디,요약
0,AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI,초상화를 '이현세 만화풍'으로 쓱쓱…만화까지 파고든 AI\nTV조선 뉴스\nPlay...,2024-05-26 19:19:00,26b32242-93f2-4472-9784-9b3b3cb622f7,
1,AI,"팻 겔싱어 인텔 CEO, 6월 방한 '무산'",[서울=뉴시스]인텔 비전 2024에서 팻 겔싱어 인텔 CEO가 인텔 가우디 3 가속...,2024-05-26 19:17:00,e4ad7804-79ca-44fd-8a63-4774ced95fa4,
2,AI,고준위 특별법-모성3법 등 민생-경제 법안 줄폐기 ‘빨간불’,ⓒ News1\n21대 국회 임기 만료가 3일 남은 가운데 여야는 민생-경제 법안...,2024-05-26 19:12:00,b7ecb4cb-0ab5-4f4b-ad29-ff31319b78eb,
3,AI,위기의 삼성 반도체…초격차 변화 예고,새수장 앞세워 대대적 전략 수정\n엔비디아 검증 탈락설 즉각 반박\n삼성전자가 인공...,2024-05-26 19:11:00,163ca46d-824e-4a65-b2b0-37839e2c6a62,
4,AI,"안철수 ""인문학 AI로 古書 복원하면 K콘텐츠 만들 소재 쏟아질 것""",22대 국회 이것만큼은\n안철수 국민의힘 분당갑 당선인\n\n최근 복원된 승정원일기...,2024-05-26 19:10:00,87e855e2-6083-4180-bada-de21d9e2e01f,
5,부동산,[안종범의 정책진단] 부자증세라는 포퓰리즘,부자 정의 명확지 않은채\n무조건 ‘돈 더 많이 내라’\n세금 초과 부담 최소화를\...,2024-05-26 19:15:00,169fe9a4-735a-40de-90ba-4b20edad015c,
6,부동산,"민주 ""'구하라법' 등 민생법안 처리 위해 법사위 열자""","""단 10건이라도 처리해야…국회의 당연한 책무""\n더불어민주당이 26일 국민의힘을 ...",2024-05-26 19:06:00,ae8fc34d-fa53-41af-b3ab-5eb0332daa8f,
7,부동산,"정부 평가기준 내놓자 바로 채점표 등장... 분당·평촌 ""결과 지켜보자"" 거래...",선도지구 선정 앞둔 1기 신도시\n소형-대형 면적따라 유불리 갈려\n재건축 주민동의...,2024-05-26 19:04:00,11b84e04-2a62-4155-b7e3-3b40686dd721,
8,부동산,"美 연착륙 할까, M7 넘을 종목은…'투자의 심장' 뉴욕서 해답 찾는다",6월 3~5일 한경 글로벌마켓 콘퍼런스 NYC 2024\n\n맥베이 KKR CIO가...,2024-05-26 19:02:00,cb98f4c8-76db-4d7d-a0e5-41ed164ddf94,
9,부동산,중소·중견기업 '가업 승계' 위해 상속·증여 공제 늘리나,"정부, 성장사다리 대책 추진\n승계 관련 세제 포함도 검토\n부동산 등 자산 이전은...",2024-05-26 19:01:00,ff863af0-9d01-4454-a0ff-72db790fb697,


In [42]:
import snowflake.connector
from snowflake.connector.pandas_tools import pd_writer
from snowflake.connector.pandas_tools import write_pandas

# Snowflake 연결
snow_conn = snowflake.connector.connect(
    user=os.getenv('user'),
    password=os.getenv('password'),
    account=os.getenv('account'),
    role=os.getenv('role'),
    warehouse=os.getenv('warehouse'),
    database=os.getenv('database'),
    schema=os.getenv('schema')
)
snowCur = snow_conn.cursor()

write_pandas(snow_conn, test_df, 'NAVER_NEWS', auto_create_table=True, overwrite=True)

snowCur.execute("ALTER TABLE NEWS.CRAWLING_DATA.NAVER_NEWS SET COMMENT='네이버뉴스 크롤링 데이터'") 